In [2]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sqlalchemy as sa

import database as db
from images import copy_images, get_list_of_images_invalid, save_metadata
from models import County, DataTrustedIdentifier, Image
from unaccent import unaccent

In [4]:
engine, session=db.connect()
engine.echo=False

2023-03-30 10:24:17,066 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-30 10:24:17,066 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 10:24:17,067 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-30 10:24:17,068 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-30 10:24:17,068 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-30 10:24:17,069 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
query = session.query(County).filter(County.uf.__eq__('PR')).distinct().all()

list_uf = [unaccent(sa.func.lower(q.uf)) for q in query]
list_state = [unaccent(sa.func.lower(q.state)) for q in query]
list_county = [unaccent(sa.func.lower(q.county)) for q in query]

uf_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_uf)
state_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.state_province)).in_(list_state)
county_unaccented_lower = unaccent(sa.func.lower(DataTrustedIdentifier.county)).in_(list_county)

In [6]:
list_minimum_image=[5, 10, 20]
list_color=['rgb', 'grayscale']
list_image_size=['256', '400', '512']
path_out = '/home/xandao/Imagens/pr_dataset'
list_images_invalid=get_list_of_images_invalid()
list_images_invalid=list_images_invalid['barcode']
list_images_invalid

['INPA0248526',
 'INPA0248523',
 'INPA0248528',
 'NY01421926_01',
 'NY01421575_01',
 'HUFSJ001689_v00',
 'HUFSJ001133_v00',
 'HUFSJ002198_v00',
 'HUFSJ003255_v00',
 'HVASF000487_v01',
 'HUFSJ001689_v01',
 'INPA0019084_nd',
 'INPA0022379_nd',
 'INPA0032742_nd',
 'INPA0023115',
 'NL-U1484137',
 'NY01397568_01',
 'INPA0012286',
 'INPA0146998']

In [7]:
for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            list_level = []
            list_path_images_final = []
            list_seq_final = []
            list_count_samples = []

            level=DataTrustedIdentifier.specific_epithet_trusted
            out=os.path.join(path_out, color.upper(), level.name, image_size, str(minimum_image))

            condition = sa.and_(DataTrustedIdentifier.country_trusted == 'Brasil',
                                level.is_not(None),
                                sa.or_(uf_unaccented_lower,
                                       state_unaccented_lower,
                                       county_unaccented_lower))

            columns = [level,
                       sa.func.array_agg(DataTrustedIdentifier.seq).label('list_seq')]
            query = session.query(*columns) \
                .filter(condition) \
                .distinct() \
                .group_by(level) \
                .order_by(level) \
                .having(sa.func.count(level) >= minimum_image) \
                .all()

            print('color: %s image_size: %s minimum_image: %d' % (color.upper(), image_size, minimum_image))
            for q in query:
                qzao = session.query(Image.seq_id, sa.func.array_agg(Image.path_segmented).label('list_path_segmented')) \
                    .filter(sa.and_(Image.seq_id.in_(q.list_seq),
                                    Image.color_mode.__eq__(color.upper()),
                                    Image.height.__eq__(image_size),
                                    Image.width.__eq__(image_size),
                                    sa.not_(Image.filename.in_(list_images_invalid)),
                                    sa.not_(Image.filename.ilike('%_v0%')),
                                    sa.not_(Image.filename.like('%_e%')),
                                    sa.not_(Image.filename.like('%_nd%')),
                                    ))\
                    .group_by(Image.seq_id)\
                    .all()

                list_one_image_per_seq = []
                list_seq_one_image = []
                for qzinho in qzao:
                    list_seq_one_image.append(qzinho.seq_id)
                    list_one_image_per_seq.append(sorted(qzinho.list_path_segmented)[0])

                if len(list_one_image_per_seq) >= minimum_image:
                    list_count_samples.append(len(list_one_image_per_seq))
                    list_level.append(q.specific_epithet_trusted)
                    list_path_images_final.append(list_one_image_per_seq)
                    list_seq_final.append(list_seq_one_image)

            copy_images(list_level, list_path_images_final, out)
            save_metadata(list_count_samples, list_level, list_path_images_final, list_seq_final, out, session)

/tmp/ipykernel_39400/3574502660.py:20: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/20/cprf)
  query = session.query(*columns) \


color: RGB image_size: 256 minimum_image: 5
total of levels: 57 total of images: 1389


,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,34,"[9674, 9927, 10002, 10051, 10094, 10468, 10676..."
1,alata,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,18,"[9865, 9884, 9911, 10134, 10178, 10271, 10380,..."
2,amalago,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,68,"[7707, 7723, 8236, 8767, 9678, 9698, 9888, 993..."
3,amplum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,6,"[10065, 10655, 11913, 11920, 11923, 12939]"


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,9735,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,13064,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
3,9821,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


color: RGB image_size: 256 minimum_image: 10
total of levels: 36 total of images: 1257


...

,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,34,"[9674, 9927, 10002, 10051, 10094, 10468, 10676..."
1,alata,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,18,"[9865, 9884, 9911, 10134, 10178, 10271, 10380,..."
2,amalago,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,68,"[7707, 7723, 8236, 8767, 9678, 9698, 9888, 993..."
3,arboreum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,37,"[8834, 9710, 9735, 9749, 9765, 9776, 9842, 991..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,9735,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,13064,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
3,9821,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


color: GRAYSCALE image_size: 512 minimum_image: 20
total of levels: 23 total of images: 1084


,levels,paths,count,seq
0,aduncum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,34,"[9674, 9927, 10002, 10051, 10094, 10468, 10676..."
1,amalago,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,68,"[7707, 7723, 8236, 8767, 9678, 9698, 9888, 993..."
2,arboreum,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,37,"[8834, 9710, 9735, 9749, 9765, 9776, 9842, 991..."
3,blanda,[/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd...,25,"[8410, 9694, 9885, 9990, 10083, 10122, 10559, ..."


,seq,genus,specific_epithet,genus_trusted,specific_epithet_trusted,urls
0,9735,Piper,arboreum,Piper,arboreum,[https://storage.googleapis.com/cria-zoomify/h...
1,7333,Piper,corcovadense,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
2,13064,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...
3,9821,Piper,corcovadensis,Piper,grazielae,[https://storage.googleapis.com/cria-zoomify/h...


In [8]:
session.close()
engine.dispose()